In [55]:
# Having jupiter comunicating with django
import setup_django
setup_django.init()

In [56]:
import pandas as pd
from django.conf import settings
from movie_recommendation_engine.playlists.models import MovieProxy
from movie_recommendation_engine.ratings.models import Rating
import os

#LINKS_SMALL_CSV = settings.DATA_DIR / 'links_small.csv'
#LINKS_SMALL_CSV.exists()

LINKS_SMALL_CSV = os.path.join(settings.DATA_DIR, "links_small.csv")

In [57]:
qs = Rating.objects.all()
missing_movie_ids = []
for instance in qs:
    if instance.content_object is None:
        missing_movie_ids.append(instance.object_id)

        
_total = len(missing_movie_ids)
total_missing = list(set(missing_movie_ids))


In [58]:
print(len(total_missing), _total, qs.count())

56 342 100005


In [59]:
links_df = pd.read_csv(LINKS_SMALL_CSV)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [60]:
ms_df = links_df.copy()[links_df.movieId.isin(total_missing)]
ms_df.head()

,movieId,imdbId,tmdbId
607,720,118114,NaN
608,721,114103,NaN
910,1133,111357,NaN
2282,2851,81454,NaN
3246,4051,56600,NaN


In [61]:
ms_df.shape[0] == len(total_missing)

True

In [62]:
def enrich_imdb_col(val):
    val = str(val)
    if len(val) == 7:
        val = f"tt{val}"
        return val
    if len(val) == 6:
        val = f"tt0{val}"
        return val
    if len(val) == 5:
        val = f"tt00{val}"
        return val
    return val

In [63]:
ms_df['tt'] = ms_df['imdbId'].apply(enrich_imdb_col)
# ms_df.to_csv('')
ms_df.head()

,movieId,imdbId,tmdbId,tt
607,720,118114,NaN,tt0118114
608,721,114103,NaN,tt0114103
910,1133,111357,NaN,tt0111357
2282,2851,81454,NaN,tt0081454
3246,4051,56600,NaN,tt0056600


In [64]:
#MOVIES_CSV = settings.DATA_DIR / 'movies_metadata.csv'
#MOVIES_CSV.exists()

MOVIES_CSV = os.path.join(settings.DATA_DIR, "movies_metadata.csv")

In [65]:
movies_cols = ['title', 'overview', 'release_date', 'imdb_id']
movies_df = pd.read_csv(MOVIES_CSV, usecols=movies_cols)
movies_df.head()

,imdb_id,overview,release_date,title
0,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [66]:
missing_movies_df = ms_df.merge(movies_df, left_on='tt', right_on='imdb_id')
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title
0,4470,94675,2.0,tt0094675,tt0094675,Taisto Kasurinen is a Finnish coal miner whose...,1988-10-21,Ariel
1,6377,266543,12.0,tt0266543,tt0266543,"Nemo, an adventurous young clownfish, is unexp...",2003-05-30,Finding Nemo
2,7502,185906,191903.0,tt0185906,tt0185906,Drawn from interviews with survivors of Easy C...,2001-09-09,Band of Brothers
3,7587,62229,5511.0,tt0062229,tt0062229,Hitman Jef Costello is a perfectionist who alw...,1967-10-25,Le Samouraï
4,7587,62229,5511.0,tt0062229,tt0062229,Hitman Jef Costello is a perfectionist who alw...,1967-10-25,Le Samouraï


In [67]:
missing_movies_df['id'] = missing_movies_df['movieId']
missing_movies_df['id_alt'] = missing_movies_df['tmdbId'].apply(lambda x: str(int(x)))
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title,id,id_alt
0,4470,94675,2.0,tt0094675,tt0094675,Taisto Kasurinen is a Finnish coal miner whose...,1988-10-21,Ariel,4470,2
1,6377,266543,12.0,tt0266543,tt0266543,"Nemo, an adventurous young clownfish, is unexp...",2003-05-30,Finding Nemo,6377,12
2,7502,185906,191903.0,tt0185906,tt0185906,Drawn from interviews with survivors of Easy C...,2001-09-09,Band of Brothers,7502,191903
3,7587,62229,5511.0,tt0062229,tt0062229,Hitman Jef Costello is a perfectionist who alw...,1967-10-25,Le Samouraï,7587,5511
4,7587,62229,5511.0,tt0062229,tt0062229,Hitman Jef Costello is a perfectionist who alw...,1967-10-25,Le Samouraï,7587,5511


In [68]:
final_df = missing_movies_df.copy()[['id', 'id_alt', 'title']]
final_df['id_alt'] = final_df['id_alt'].astype(str)
final_df.head()

,id,id_alt,title
0,4470,2,Ariel
1,6377,12,Finding Nemo
2,7502,191903,Band of Brothers
3,7587,5511,Le Samouraï
4,7587,5511,Le Samouraï


In [69]:
alt_id_list = final_df['id_alt'].to_list()

In [70]:
#movie_qs = MovieProxy.objects.filter(id__in=alt_id_list)
movie_qs.count()

988

In [71]:
from django.forms.models import model_to_dict

In [72]:
movies_qs = MovieProxy.objects.filter(id__in=alt_id_list)
for obj in movies_qs:
    data = final_df.copy()[final_df['id_alt'] == str(obj.id)]
    if data.shape[0] == 1:
        og_model_data = model_to_dict(obj)
        update_data = data.to_dict('records')[0]
        if obj.title == update_data.get('title'): 
            #print(og_model_data)
            og_model_data['id'] = update_data['id']
            new_model_data = {**og_model_data}
            #print(new_model_data)
            #obj.delete()
            #MovieProxy.objects.create(**new_model_data)
            obj.id =new_model_data["id"]
            obj.save()

In [ ]:
# movies_qs = MovieProxy.objects.filter(id__in=alt_id_list)
# for obj in movies_qs:
#     data = final_df.copy()[final_df['id_alt'] == str(obj.id)]
#     if data.shape[0] == 1:
#         og_model_data = model_to_dict(obj)
#         update_data = data.to_dict('records')[0]
#         if obj.title == update_data.get('title'): 
#             #print(og_model_data)
#             og_model_data['id'] = update_data['id']
#             new_model_data = {**og_model_data}
#             #print(new_model_data)
#             obj.delete()
#             #MovieProxy.objects.create(**new_model_data)
#             MovieProxy.objects.create(
#                 id=new_model_data["id"],
#                 parent=new_model_data["parent"], 
#                 category=new_model_data["category"], 
#                 order=new_model_data["order"], 
#                 title=new_model_data["title"], 
#                 type=new_model_data["type"], 
#                 overview=new_model_data["overview"], 
#                 release_date=new_model_data["release_date"],  
#                 slug=new_model_data["slug"], 
#                 #video=new_model_data["video"], 
#                 active=new_model_data["active"], 
#                 state=new_model_data["state"], 
#                 publish_timestamp=new_model_data["publish_timestamp"], 
#                 rating_last_updated=new_model_data["rating_last_updated"], 
#                 rating_count=new_model_data["rating_count"], 
#                 rating_avg=new_model_data["rating_avg"], 
#             )

In [ ]:
from movie_recommendation_engine.ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()